In [1]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## Load the Dataset
data = gutenberg.raw('shakespeare-hamlet.txt')
## Save the dataset in the file
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\bhask\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Data Preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## Load the dataset
with open('hamlet.txt','r') as file:
    text = file.read().lower()

## Tokenize the text - Creating indexes for the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

4818

In [3]:
#tokenizer.word_index

In [4]:
## Create Input Sequence
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range (1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
# input_sequences

In [6]:
## Pad sequences (To make all the sentences of equal length)
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [7]:
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [8]:
# input_sequences

In [9]:
## Create the Predictors and Label
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [10]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [11]:
y ##indexes

array([ 687,    4,   45, ..., 1047,    4,  193])

In [12]:
## Convert y into categories, as there might be some sequence of the index repeated
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
## Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [14]:
## Define early stopping
# from tensorflow.keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [15]:
## Train our LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

## Define the model
model = Sequential()
model.add(Embedding(input_dim = total_words,output_dim= 100,input_shape=(max_sequence_len,)))  # Ensures the model is built
model.add(LSTM(150,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation = "softmax"))

# Define the model
# model = Sequential([
#     Embedding(input_dim=total_words, output_dim=100,input_shape=(max_sequence_len,)),  # No input_shape here
#     LSTM(150, return_sequences=True),
#     Dropout(0.2),
#     LSTM(100),
#     Dense(total_words, activation="softmax")

# ])

## Compile the Model
model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
model.summary()

c:\Users\bhask\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 14, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
## Train the model
history = model.fit(x_train,y_train,epochs=200,validation_data=(x_test,y_test),verbose=1)

## Train the model
# history = model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1, callbacks=[early_stopping])

Epoch 1/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.0311 - loss: 7.1652 - val_accuracy: 0.0383 - val_loss: 6.7011
Epoch 2/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0332 - loss: 6.4973 - val_accuracy: 0.0474 - val_loss: 6.7889
Epoch 3/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0429 - loss: 6.3332 - val_accuracy: 0.0536 - val_loss: 6.8110
Epoch 4/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0513 - loss: 6.1766 - val_accuracy: 0.0542 - val_loss: 6.8372
Epoch 5/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0541 - loss: 6.0624 - val_accuracy: 0.0571 - val_loss: 6.8648
Epoch 6/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0558 - loss: 5.9499 - val_accuracy: 0.0602 - val_loss: 6.9121
Epoch 7/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0654 - loss: 5.7781 - val_accuracy: 0.0643 - val_loss: 6.9530
Epoch 8/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0711 - loss: 5.6737 -

In [17]:
# Function to predict the next word
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len):]  # Ensure the sequence length matches max_sequence_len
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted, axis = 1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [18]:
input_text = "Mar. Horatui saies, 'tis but our"
print(f"Input Text : {input_text}")
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word prediction : {next_word}")

Input Text : Mar. Horatui saies, 'tis but our
Next word prediction : fantasie


In [20]:
import pickle
## Save the model
model.save("Next_word_lstm.h5")
## Save the tokenizer

with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)